##Import the necessaarty libraries

In [2]:
pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.4 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import plotly.express as px
import streamlit as st  # Optional, only if building the web app
import warnings
warnings.filterwarnings('ignore')


#Load the dataset

In [4]:
data = pd.read_csv('/content/Market_Basket_Optimisation.csv')


In [5]:
data.tail()

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
7495,butter,light mayo,fresh bread,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7496,burgers,frozen vegetables,eggs,french fries,magazines,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7497,chicken,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7498,escalope,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7499,eggs,frozen smoothie,yogurt cake,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
data.head()

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
0,burgers,meatballs,eggs,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chutney,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,turkey,avocado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,mineral water,milk,energy bar,whole wheat rice,green tea,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,low fat yogurt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Data exploration

In [8]:
data.isnull().sum()

,0
shrimp,0
almonds,1754
avocado,3112
vegetables mix,4156
green grapes,4972
whole weat flour,5637
yams,6132
cottage cheese,6520
energy drink,6847
tomato juice,7106


In [11]:
data.replace(['0', 'None', 'missing'], '', inplace=True)


In [13]:
transactions = []

for _, row in data.iterrows():
    cleaned = [item for item in row if pd.notna(item) and str(item).strip() != '']
    transactions.append(cleaned)


In [14]:
data.columns

Index(['shrimp', 'almonds', 'avocado', 'vegetables mix', 'green grapes',
       'whole weat flour', 'yams', 'cottage cheese', 'energy drink',
       'tomato juice', 'low fat yogurt', 'green tea', 'honey', 'salad',
       'mineral water', 'salmon', 'antioxydant juice', 'frozen smoothie',
       'spinach', 'olive oil'],
      dtype='object')

In [16]:
data.describe(include='all')

,shrimp,almonds,avocado,vegetables mix,green grapes,whole weat flour,yams,cottage cheese,energy drink,tomato juice,low fat yogurt,green tea,honey,salad,mineral water,salmon,antioxydant juice,frozen smoothie,spinach,olive oil
count,7500,5746,4388,3344,2528,1863,1368,980,653,394,255,153,86,46,24,7,3,3,2,0.0
unique,115,117,115,114,110,106,102,98,88,80,66,50,42,28,18,7,2,2,2,NaN
top,mineral water,mineral water,mineral water,mineral water,green tea,french fries,green tea,green tea,green tea,green tea,low fat yogurt,green tea,green tea,green tea,magazines,chocolate,frozen smoothie,protein bar,mayonnaise,NaN
freq,577,484,375,201,153,107,96,67,57,31,21,14,8,4,3,1,2,2,1,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   shrimp             7500 non-null   object 
 1   almonds            5746 non-null   object 
 2   avocado            4388 non-null   object 
 3   vegetables mix     3344 non-null   object 
 4   green grapes       2528 non-null   object 
 5   whole weat flour   1863 non-null   object 
 6   yams               1368 non-null   object 
 7   cottage cheese     980 non-null    object 
 8   energy drink       653 non-null    object 
 9   tomato juice       394 non-null    object 
 10  low fat yogurt     255 non-null    object 
 11  green tea          153 non-null    object 
 12  honey              86 non-null     object 
 13  salad              46 non-null     object 
 14  mineral water      24 non-null     object 
 15  salmon             7 non-null      object 
 16  antioxydant juice  3 non

#Handle the duplicates

In [20]:
data.duplicated().sum()

np.int64(2325)

In [24]:
cleaned_transactions = []

for _, row in data.iterrows():
    items = [str(item).strip().lower() for item in row if pd.notna(item)]
    items = list(set(items))  # removes duplicates within a transaction
    cleaned_transactions.append(items)


In [23]:
data.duplicated().sum()

np.int64(2325)

In [25]:
duplicates = data.duplicated().sum()
print(duplicates)


2325


In [26]:
print("Before:", len(data))
print("Duplicates:", data.duplicated().sum())


Before: 7500
Duplicates: 2325


#Outliers

In [28]:
cleaned_transactions = []

for _, row in data.iterrows():
    items = [str(item).strip().lower() for item in row if pd.notna(item)]
    items = list(set(items))  # remove duplicate items
    if len(items) > 1:        # optional: remove tiny baskets
        cleaned_transactions.append(items)


#Emcoding(TransactionEncoder)

In [29]:
from mlxtend.preprocessing import TransactionEncoder
import pandas as pd


# Step 1: Fit and transform using TransactionEncoder
te = TransactionEncoder()
te_array = te.fit_transform(transactions)

# Step 2: Convert to DataFrame
df_encoded = pd.DataFrame(te_array, columns=te.columns_)
print(df_encoded)


       asparagus  almonds  antioxydant juice  asparagus  avocado  babies food  \
0          False    False              False      False    False        False   
1          False    False              False      False    False        False   
2          False    False              False      False     True        False   
3          False    False              False      False    False        False   
4          False    False              False      False    False        False   
...          ...      ...                ...        ...      ...          ...   
7495       False    False              False      False    False        False   
7496       False    False              False      False    False        False   
7497       False    False              False      False    False        False   
7498       False    False              False      False    False        False   
7499       False    False              False      False    False        False   

      bacon  barbecue sauce

##Run Apriori:

In [30]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(df_encoded, min_support=0.01, use_colnames=True)
print(frequent_itemsets)


      support                                 itemsets
0    0.020267                                (almonds)
1    0.033200                                (avocado)
2    0.010800                         (barbecue sauce)
3    0.014267                              (black tea)
4    0.011467                             (body spray)
..        ...                                      ...
254  0.011067       (milk, mineral water, ground beef)
255  0.017067  (ground beef, mineral water, spaghetti)
256  0.015733         (milk, mineral water, spaghetti)
257  0.010267    (olive oil, mineral water, spaghetti)
258  0.011467     (pancakes, mineral water, spaghetti)

[259 rows x 2 columns]


#Generate association rules

In [31]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


                   antecedents      consequents   support  confidence  \
0                    (avocado)  (mineral water)  0.011467    0.345382   
1                    (burgers)           (eggs)  0.028800    0.330275   
2                       (cake)  (mineral water)  0.027467    0.338816   
3                    (cereals)  (mineral water)  0.010267    0.398964   
4                    (chicken)  (mineral water)  0.022800    0.380000   
..                         ...              ...       ...         ...   
58           (milk, spaghetti)  (mineral water)  0.015733    0.443609   
59  (olive oil, mineral water)      (spaghetti)  0.010267    0.373786   
60      (olive oil, spaghetti)  (mineral water)  0.010267    0.447674   
61   (pancakes, mineral water)      (spaghetti)  0.011467    0.339921   
62       (pancakes, spaghetti)  (mineral water)  0.011467    0.455026   

        lift  
0   1.449559  
1   1.837585  
2   1.422002  
3   1.674442  
4   1.594852  
..       ...  
58  1.861817  
59 

In [32]:
# Sort strongest rules
strong_rules = rules[(rules['lift'] > 1) & (rules['confidence'] >= 0.5)]
strong_rules = strong_rules.sort_values(by='lift', ascending=False)


In [33]:
# Generate rules from frequent itemsets
from mlxtend.frequent_patterns import association_rules

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.3)

# Filter strong rules
strong_rules = rules[(rules['lift'] > 1) & (rules['confidence'] > 0.5)]

# Display
print(strong_rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


            antecedents      consequents   support  confidence      lift
45  (eggs, ground beef)  (mineral water)  0.010133    0.506667  2.126469
54  (milk, ground beef)  (mineral water)  0.011067    0.503030  2.111207
